# Credit Risk Pipeline Quickstart

This notebook runs the **Unified Risk Pipeline** end-to-end on the bundled synthetic dataset.
The sample includes stratified monthly observations, calibration hold-outs, stage-2 data, and a future scoring batch
so each major step can be validated quickly.


## 0. Environment setup

This cell ensures `risk-pipeline` 0.4.1 is installed from the GitHub `development` branch.
Restart the kernel and rerun after refreshing the environment.


In [ ]:
import importlib
import importlib.metadata as metadata
import subprocess
import sys

TARGET_VERSION = "0.4.1"
GIT_SPEC = "risk-pipeline[ml,notebook] @ git+https://github.com/selimoksuz/risk-model-pipeline.git@development"
PREREQ_PACKAGES = [
    "numba==0.59.1",
    "llvmlite==0.42.0",
    "scipy==1.11.4",
    "pandas==2.3.2",
    "tsfresh==0.20.1",
    "matrixprofile==1.1.10",
    "shap==0.48.0",
    "stumpy==1.13.0",
]

def _parse_version(value: str):
    parts = []
    for part in value.split('.'):
        if not part.isdigit():
            break
        parts.append(int(part))
    return tuple(parts)

def _run_pip(args):
    subprocess.check_call([
        sys.executable,
        "-m",
        "pip",
        "install",
        "--no-cache-dir",
        "--upgrade",
        "--force-reinstall",
        *args,
    ])

def _install_prerequisites():
    print(f"Installing prerequisite stack: {', '.join(PREREQ_PACKAGES)}")
    _run_pip(PREREQ_PACKAGES)

def _sanity_check():
    import shap  # noqa: F401
    from llvmlite import binding as _ll_binding
    _ = _ll_binding.ffi.lib
    from numba import njit

    @njit
    def _probe(x):
        return x + 1

    assert _probe(1) == 2

def _tsfresh_smoke_test():
    import pandas as pd
    from tsfresh import extract_features
    from tsfresh.feature_extraction import EfficientFCParameters

    data = pd.DataFrame(
        {
            "id": ["a", "a", "a", "b", "b", "b"],
            "time": [0, 1, 2, 0, 1, 2],
            "value": [1.0, 2.0, 3.0, 4.0, 9.0, 16.0],
        }
    )
    features = extract_features(
        data,
        column_id="id",
        column_sort="time",
        column_value="value",
        default_fc_parameters=EfficientFCParameters(),
        disable_progressbar=True,
        n_jobs=0,
    )
    if not any("entropy" in col for col in features.columns):
        raise RuntimeError("tsfresh smoke test did not produce entropy features")

def ensure_risk_pipeline():
    try:
        importlib.import_module("risk_pipeline")
        installed = metadata.version("risk-pipeline")
        if _parse_version(installed) < _parse_version(TARGET_VERSION):
            raise ModuleNotFoundError(f"risk-pipeline {installed} < {TARGET_VERSION}")
        print(f"risk-pipeline {installed} already installed.")
        _sanity_check()
        _tsfresh_smoke_test()
    except Exception as exc:
        print(f"risk-pipeline import failed: {exc}")
        try:
            _install_prerequisites()
            print(f"Attempting GitHub install: {GIT_SPEC}")
            _run_pip([GIT_SPEC])
            print("GitHub install succeeded.")
            raise SystemExit("Installation complete. Restart the kernel and rerun this cell.")
        except subprocess.CalledProcessError as err:
            print(f"GitHub install failed: {err}")
            raise SystemExit("Installation failed. Review the errors above.")
    else:
        print("Numba/llvmlite sanity check passed.")
        print("tsfresh smoke test passed (entropy features available).")

ensure_risk_pipeline()


## 1. Imports and sample loader

The dataset ships with the package under `risk_pipeline.data.sample`.

In [ ]:
from pathlib import Path
import pandas as pd

from IPython.display import display

from risk_pipeline.core.config import Config
from risk_pipeline.unified_pipeline import UnifiedRiskPipeline
from risk_pipeline.data.sample import load_credit_risk_sample

sample = load_credit_risk_sample()
OUTPUT_DIR = Path('output/credit_risk_sample_notebook')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

dev_df = sample.development
cal_long_df = sample.calibration_longrun
cal_recent_df = sample.calibration_recent
score_df = sample.scoring_future
data_dictionary = sample.data_dictionary

dev_df.head()

## 2. Quick sanity checks

In [ ]:
dev_df['target'].value_counts(normalize=True).rename('default_rate')

In [ ]:
dev_df.groupby('snapshot_month')['target'].mean().rename('monthly_default_rate')

## 3. Configure the pipeline

The configuration below enables dual modelling (raw + WoE), Optuna (single rapid trial), balanced model selection with stability guard rails,
noise sentinel monitoring, SHAP explainability, the WoE-LI and Shao logistic challengers, and the PD-constrained risk band optimizer.
Train/Test/OOT ratios and all threshold knobs (PSI/IV/Gini/Correlation) are explicit so the notebook mirrors production-ready configuration files.

In [ ]:
cfg = Config(
    target_column='target',
    id_column='customer_id',
    time_column='app_dt',
    create_test_split=True,
    use_test_split=True,
    train_ratio=0.6,
    test_ratio=0.2,
    oot_ratio=0.2,
    stratify_test=True,
    oot_months=2,
    enable_dual=True,
    enable_tsfresh_features=True,
    tsfresh_feature_set='efficient',
    tsfresh_n_jobs=-1,
    enable_scoring=True,
    enable_stage2_calibration=True,
    output_folder=str(OUTPUT_DIR),
    selection_steps=['psi', 'univariate', 'iv', 'correlation', 'boruta', 'stepwise'],
    algorithms=[
        'logistic', 'gam', 'catboost', 'lightgbm', 'xgboost',
        'randomforest', 'extratrees', 'woe_boost', 'woe_li', 'shao', 'xbooster',
    ],
    model_selection_method='balanced',
    model_stability_weight=0.25,
    min_gini_threshold=0.45,
    max_train_oot_gap=0.08,
    psi_threshold=0.25,
    iv_threshold=0.02,
    univariate_gini_threshold=0.05,
    correlation_threshold=0.95,
    vif_threshold=5.0,
    woe_binning_strategy='iv_optimal',
    use_optuna=True,
    n_trials=1,
    optuna_timeout=120,
    hpo_method='optuna',
    hpo_trials=1,
    hpo_timeout_sec=120,
    use_noise_sentinel=True,
    calculate_shap=True,
    shap_sample_size=500,
    risk_band_method='pd_constraints',
    n_risk_bands=8,
    risk_band_min_bins=7,
    risk_band_max_bins=10,
    risk_band_micro_bins=1000,
    risk_band_min_weight=0.05,
    risk_band_max_weight=0.30,
    risk_band_hhi_threshold=0.15,
    risk_band_binomial_pass_weight=0.85,
    risk_band_alpha=0.05,
    risk_band_pd_dr_tolerance=1e-4,
    risk_band_max_iterations=100,
    risk_band_max_phase_iterations=50,
    risk_band_early_stop_rounds=10,
    calibration_stage1_method='isotonic',
    calibration_stage2_method='lower_mean',
    random_state=42,
)
cfg.model_type = 'all'


## 4. Run the unified pipeline

In [ ]:
pipe = UnifiedRiskPipeline(cfg)
results = pipe.fit(
    dev_df,
    data_dictionary=data_dictionary,
    calibration_df=cal_long_df,
    stage2_df=cal_recent_df,
    score_df=score_df,
)

## 5. Inspect key outputs

In [ ]:
best_model = results.get('best_model_name')
model_scores = results.get('model_results', {}).get('scores', {})
print(f'Best model: {best_model}')
pd.DataFrame(model_scores).T

In [ ]:
feature_report = pipe.reporter.reports_.get('features')
feature_report.head() if feature_report is not None else 'No feature report available.'

In [ ]:
calibration_report = pipe.reporter.reports_.get('calibration')
calibration_report

In [ ]:
risk_band_source = pipe.results_.get('risk_bands')
risk_bands_table = pipe.reporter.generate_risk_band_report(risk_band_source) if risk_band_source else None
risk_bands_summary = pipe.reporter.reports_.get('risk_bands_summary_table')
risk_bands_tests = pipe.reporter.reports_.get('risk_bands_tests')

display(risk_bands_table if isinstance(risk_bands_table, pd.DataFrame) and not risk_bands_table.empty else 'No risk band table available.')
display(risk_bands_summary if isinstance(risk_bands_summary, pd.DataFrame) and not risk_bands_summary.empty else 'No risk band summary available.')
display(risk_bands_tests if isinstance(risk_bands_tests, pd.DataFrame) and not risk_bands_tests.empty else 'No binomial test results available.')


## 6. Generated files

In [ ]:
sorted(p.relative_to(OUTPUT_DIR.parent) for p in OUTPUT_DIR.glob('**/*') if p.is_file())

## 7. XBooster scorecard

In [ ]:
xbooster_artifacts = results.get('model_results', {}).get('interpretability', {}).get('XBooster', {})
if isinstance(xbooster_artifacts, dict):
    scorecard_df = xbooster_artifacts.get('scorecard_points')
    warnings = xbooster_artifacts.get('warnings')
    display_obj = scorecard_df.head() if hasattr(scorecard_df, 'head') else xbooster_artifacts
else:
    warnings = None
    display_obj = 'No XBooster artifacts available.'
print('Warnings:', warnings if warnings else 'None')
display_obj

## 8. Automating via script

`examples/quickstart_demo.py` mirrors the steps above so the flow can be validated headless
(e.g. in CI pipelines).